# 2 - Labelling view datasets with pretrained model

In this tutorial, we will work with a view dataset that we created in the previous tutorial, and label that with web-image trained models. Lots of the code in this notebook is just going to be boilerplate to use our pretrained models as label extractors, but the core idea is that we are taking a "view dataset", defined in the previous tutorial as a combination of RGB, depth, and confidence information, and convert that to point-wise dataset that contains the XYZ position of a point, a semantic encoding of the label of whatever is at that point, and a vision-language model (aka CLIP) embedding of whatever object is at that point.

Conceretely, the dataset we create will have a `__getitem__` method that returns a dictionary with the following keys:
```python
{
    "xyz": torch.Tensor, # (3,) tensor of XYZ position of point
    "rgb": torch.Tensor, # (3,) tensor of RGB color of point
    "label": torch.Tensor, # (1,) tensor ID of semantic label of point
    "instance": torch.Tensor, # (1,) tensor ID of instance label of point, if exists, else -1
    "img_idx": torch.Tensor, # (1,) tensor ID of image that point was sampled from, or the bounding box that this point is a part of.
    "distance": torch.Tensor, # (1,) tensor of distance from point to camera
    "clip_vector": torch.Tensor, # (512,) or (768,), tensor of CLIP-text or Sentence-BERT embedding of point
    "clip_image_vector": torch.Tensor, # (512,) or (768,), tensor of CLIP-image embedding of point
    "semantic_weight": torch.Tensor, # (1,) tensor of weight of semantic label of point; generally the confidence of the semantic label coming from the detector.
}
```

Since this class uses a lot of boilerplate code, we are not reproducing it here, but you can find the code for our pretrained model based labeller in `dataloaders/real_dataset.py`

In [3]:
import sys
sys.path.append('..')

import torch
from dataloaders import R3DSemanticDataset, DeticDenseLabelledDataset

# Load the dataset
# If you are following up after tutorial 1
# dataset = torch.load("../nyu_views.pth")
# Otherwise, create from scratch.
dataset = R3DSemanticDataset("../nyu.r3d")

Calculating global XYZs: 100%|████████████████████████████████████████████████████████████| 757/757 [00:38<00:00, 19.84it/s]


NameError: name 'DataLoader' is not defined

In [4]:
from torch.utils.data import Dataset, DataLoader, Subset

dataloader = DataLoader(dataset, batch_size=1, shuffle=False, pin_memory=False)

In [8]:
import tqdm
import einops


for idx, data_dict in tqdm.tqdm(enumerate(dataloader), total=len(dataset), desc="Calculating Detic features"):
    rgb = einops.rearrange(data_dict["rgb"][..., :3], "b h w c -> b c h w")
    xyz = data_dict["xyz_position"]

    print(rgb.shape)
    print(xyz.shape)

Calculating Detic features:   6%|███▎                                                     | 44/757 [00:00<00:01, 438.44it/s]

torch.Size([1, 3, 960, 720])
torch.Size([1, 546813, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 550386, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 547062, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 546538, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 545127, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 549144, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 545976, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 546555, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 541935, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 537852, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 531846, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 525152, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 528725, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 529408, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 524723, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 521285, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 519105, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 5117

Calculating Detic features:  17%|█████████▌                                              | 129/757 [00:00<00:01, 399.43it/s]

torch.Size([1, 3, 960, 720])
torch.Size([1, 596885, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 613806, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 616233, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 616532, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 616835, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 604672, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 606912, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 601262, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 604691, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 599221, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 603106, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 602714, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 604784, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 604703, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 625127, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 618241, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 625963, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 6158

Calculating Detic features:  29%|████████████████▏                                       | 218/757 [00:00<00:01, 425.69it/s]

...

torch.Size([1, 3, 960, 720])
torch.Size([1, 556273, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 566387, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 567086, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 580784, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 580484, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 570554, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 593498, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 581896, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 572337, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 566792, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 571702, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 553191, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 561035, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 561051, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 567567, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 579791, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 604283, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 5768

Calculating Detic features:  87%|████████████████████████████████████████████████▌       | 657/757 [00:01<00:00, 540.30it/s]

torch.Size([1, 3, 960, 720])
torch.Size([1, 524991, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 536028, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 535641, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 556587, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 533091, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 552799, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 530129, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 558952, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 557950, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 564827, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 560186, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 548424, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 557087, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 568522, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 547962, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 567633, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 575586, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 5516

Calculating Detic features: 100%|████████████████████████████████████████████████████████| 757/757 [00:01<00:00, 481.77it/s]

torch.Size([1, 3, 960, 720])
torch.Size([1, 591212, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 589217, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 585962, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 579991, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 592268, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 583798, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 573674, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 589037, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 587028, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 596379, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 590547, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 606813, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 597980, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 607992, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 595522, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 597476, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 589880, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 5799

In [2]:
labelled_dataset = DeticDenseLabelledDataset(
    dataset, 
    use_extra_classes=False, 
    exclude_gt_images=False, 
    use_lseg=False, 
    subsample_prob=0.01, 
    visualize_results=True, 
    detic_threshold=0.4,
    visualization_path="detic_labelled_results",
)

INFO - 2022-12-06 16:48:28,010 - SentenceTransformer - Load pretrained SentenceTransformer: all-mpnet-base-v2
INFO - 2022-12-06 16:48:29,050 - SentenceTransformer - Use pytorch device: cuda
INFO - 2022-12-06 16:48:33,391 - checkpoint - [Checkpointer] Loading from https://dl.fbaipublicfiles.com/detic/Detic_LCOCOI21k_CLIP_SwinB_896b32_4x_ft4x_max-size.pth ...
INFO - 2022-12-06 16:48:33,392 - file_io - URL https://dl.fbaipublicfiles.com/detic/Detic_LCOCOI21k_CLIP_SwinB_896b32_4x_ft4x_max-size.pth cached in /root/.torch/iopath_cache/detic/Detic_LCOCOI21k_CLIP_SwinB_896b32_4x_ft4x_max-size.pth


Loading pretrained CLIP


Calculating Detic features:   0%|                                                                   | 0/757 [00:00<?, ?it/s]/root/anaconda3/envs/cf/lib/python3.8/site-packages/torch/utils/data/_utils/collate.py:63: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /opt/conda/conda-bld/pytorch_1627336325426/work/torch/csrc/utils/tensor_numpy.cpp:143.)
  return default_collate([torch.as_tensor(b) for b in batch])


torch.Size([1, 3, 960, 720])
torch.Size([1, 546813, 3])


Calculating Detic features:   0%|                                                           | 1/757 [00:01<13:15,  1.05s/it]

...

torch.Size([611272])
torch.Size([611272, 3])
torch.Size([611272])
torch.Size([611272, 3])
torch.Size([611272])
torch.Size([611272, 3])
torch.Size([611272])
torch.Size([611272, 3])
torch.Size([611272])
torch.Size([611272, 3])
torch.Size([611272])
torch.Size([611272, 3])
torch.Size([611272])
torch.Size([611272, 3])
torch.Size([611272])
torch.Size([611272, 3])
torch.Size([611272])
torch.Size([611272, 3])
torch.Size([611272])
torch.Size([611272, 3])
torch.Size([611272])
torch.Size([611272, 3])
torch.Size([611272])
torch.Size([611272, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 615663, 3])
torch.Size([615663])
torch.Size([615663, 3])
torch.Size([615663])
torch.Size([615663, 3])
torch.Size([615663])
torch.Size([615663, 3])
torch.Size([615663])
torch.Size([615663, 3])
torch.Size([615663])
torch.Size([615663, 3])
torch.Size([615663])
torch.Size([615663, 3])
torch.Size([615663])
torch.Size([615663, 3])
torch.Size([615663])
torch.Size([615663, 3])
torch.Size([615663])
torch.Size([615663, 3])

Calculating Detic features:  18%|██████████▏                                              | 135/757 [01:20<06:58,  1.49it/s]

torch.Size([615663])
torch.Size([615663, 3])
torch.Size([615663])
torch.Size([615663, 3])
torch.Size([615663])
torch.Size([615663, 3])
torch.Size([615663])
torch.Size([615663, 3])
torch.Size([615663])
torch.Size([615663, 3])
torch.Size([615663])
torch.Size([615663, 3])
torch.Size([615663])
torch.Size([615663, 3])
torch.Size([615663])
torch.Size([615663, 3])
torch.Size([615663])
torch.Size([615663, 3])
torch.Size([615663])
torch.Size([615663, 3])
torch.Size([615663])
torch.Size([615663, 3])
torch.Size([1, 3, 960, 720])
torch.Size([1, 610737, 3])


Calculating Detic features:  18%|██████████▏                                              | 135/757 [01:20<06:09,  1.68it/s]


KeyboardInterrupt: 

## Visualizing the labelling

We can visualize the labelling with our web-image models by setting the `visualize_results=True` and `visualization_path` to a desired path, as seen in the above cell. This step is incredibly useful for debugging, and can help you understand what the model is doing. We have found that the downstream training can sometimes be suboptimal if the initial labelling is not that great. In this case, we can use the visualization to understand what the model is doing, and then if necessary, modify the set of to be more accurate.

In [ ]:
import glob
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

NUM_IMAGES = 10
TOTAL_FRAMES = glob.glob("detic_labelled_results/*.jpg")
fig, axes = plt.subplots(NUM_IMAGES, 1, figsize=(3, 3 * NUM_IMAGES))

for ax, data in zip(axes, range(NUM_IMAGES)):
    random_index = np.random.randint(0, len(TOTAL_FRAMES))
    ax.imshow(Image.open(TOTAL_FRAMES[random_index]))
    ax.axis("off")
    ax.set_title(f"Frame {random_index}")

## Save the dataloader

Finally, we save the dataloader so that we can use it on our next step for training.

In [5]:
torch.save(labelled_dataset, "../detic_labeled_dataset.pt")